In [ ]:
################################################
# New style 
################################################
import sys
import os

# Cartopy for pretty maps
# Done first because of compatibilty issues
# with ESMF
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature


workdir_ = '/glade/work/juliob'
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} ")

from myPythonTools.Utils import utils as uti
from myPythonTools.Utils import numerical_utils as nuti
from myPythonTools.Utils import AveragingUtils as Av
from myPythonTools.Utils import validation_data as Val
from myPythonTools.Utils import PlotUtil as Pu
from myPythonTools.Plotting import LatLonMaps as LL 
from myPythonTools.CASutils import filter_utils as fu 

from PyRegridding.Utils import GridUtils as GrU
from PyRegridding.Utils import MakePressures as MkP
from PyRegridding.Drivers import RegridField as RgF
from PyRegridding.Regridder import scripGen


# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
#importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(RgF)
importlib.reload(LL)

RgObs={}


In [ ]:
### Make a list of files
import yaml
#from box import Box


with open('configure_plots.yaml', 'r') as file:
    cfg = yaml.safe_load(file)

######################################################
season = cfg['plot']['season']
plotDst = cfg['plot']['DstGrid']
valData  = cfg['validation']['data']
valYears = cfg['validation']['years']


xl= ['CAM7_ne30_FMT'  , 'oldCTL'  , ] #'CAM7_ne120_FMT' ,] #'ne240x2_QxQsst']
Al =[]
for x in xl:
    exp, subd, Src, Hkey, Dst, useri = cfg[x]['name'] , cfg[x]['subdir'] , cfg[x]['SrcGrid'] , cfg[x]['Hkey'] , cfg[x]['DstGrid'] , cfg[x]['user'] 
    ymdPat = cfg[x]['ymdPat']
    print( exp, subd, Src, Hkey, Dst, useri )
    print( ymdPat , season )
    print( valData , valYears )
    
    A = uti.MakeDict4Exp( exp=exp , user=useri, subd=subd , 
                     hsPat='cam.h0a' , ymdPat=ymdPat,verbose=True, open_dataset=True )
    A['Src']=Src
    A['Dst']=Dst
    A['Hkey']=Hkey
    Al.append( A )

########################################
# Make seasonal means
########################################

for A in Al:
    print( f'{A.exp} Years {len(A.X.time) /12. }')
    
    taux_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld='TAUX',return_time=True)
    tauy_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld='TAUY',return_time=True)
    A['taux_s']=taux_s
    A['tauy_s']=tauy_s
    A['years']=years_a
    A['months']=months_a






########################################
# Regrid if needed
########################################
for A in Al:
    Hkey = A['Hkey']
    Src  = A['Src']
    taux_s = A['taux_s']
    tauy_s = A['tauy_s']
    if (Src in ('fvQxQ','latlonOxO')) and (cfg['plot']['ForceRegrid']==True):
        ForceRegrid=True
        print( f'Regridding from {Src} to {plotDst} will be forced!!!')
    else:
        ForceRegrid=False
    
    if (Hkey == 'yx' ) and (ForceRegrid==False):
        ########################
        # Option 1 from lat-lon regirrdded data
        ########################

        lon_a=A.X.lon.values
        lat_a=A.X.lat.values
        lev_a=A.X.lev.values
        zlev_a=-7.0*np.log( lev_a/1_000. )
        taux_s_yx=taux_s
        tauy_s_yx=tauy_s
   
    elif (Hkey=='c') or (ForceRegrid==True):
        ########################
        # Option 2 from unstructured SE data
        ########################
        
        Dst = plotDst
        DstInfo = GrU.gridInfo(Dst) #,Vgrid=DstVgrid)
        lat_a,lon_a = GrU.latlon( scrip= DstInfo['scrip'], Hkey=DstInfo['Hkey'] )
        lev_a=A.X.lev.values
        zlev_a=-7.0*np.log( lev_a/1_000. )
    
        RegridObj = GrU.regrid_object_lib(RgOb=RgObs, src=Src, dst=Dst)
        taux_s_yx=RgF.Horz(xfld_Src=taux_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
        tauy_s_yx=RgF.Horz(xfld_Src=tauy_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )

    curltau_s_yx =nuti.Sphere_Curl2( f_x=taux_s_yx , f_y=tauy_s_yx , lat=lat_a , lon=lon_a , wrap=True )
    
    A['taux_s_yx'] = taux_s_yx
    A['tauy_s_yx'] = tauy_s_yx
    A['curltau_s_yx'] = curltau_s_yx
    
    A['lon_yx'] = lon_a
    A['lat_yx'] = lat_a
    A['DateRange'] = f"{(A.X.time[0].values.item() ).strftime('%Y-%m-%d')}_{(A.X.time[-1].values.item() ).strftime('%Y-%m-%d')}"

In [ ]:
####
# Validation
####


valYears='*' #'2004'
val = Val.data(fld='TAUX',season=season, Years=valYears ) #, Years='1996' ) #, Years='2000' )
taux_v_yx , zlev_v,lat_v,lon_v, years_Tv = val['aa'], val['lev'], val['lat'], val['lon'], val['years']
val = Val.data(fld='TAUY',season=season, Years=valYears ) #, Years='1996' ) #, Years='2000' )
tauy_v_yx , zlev_v,lat_v,lon_v, years_Tv = val['aa'], val['lev'], val['lat'], val['lon'], val['years']

taux_v_yx = np.nan_to_num(taux_v_yx , nan=0)
tauy_v_yx = np.nan_to_num(tauy_v_yx , nan=0)

val = Val.data(fld='WIND_STRESS_CURL',season=season, Years=valYears ) #, Years='1996' ) #, Years='2000' )
curltau_v_yx , zlev_v,lat_v,lon_v, years_Tv = val['aa'], val['lev'], val['lat'], val['lon'], val['years']

if (cfg['plot']['ForceRegrid']==True):
    dummy=scripGen.latlon_to_scrip( longitudes=lon_v, latitudes=lat_v, file_out='/glade/work/juliob/GridFiles/tmp/tmp_validation_scrip.nc' )
    Src='validation'
    Dst=plotDst
    RegridObj = GrU.regrid_object_lib(RgOb=RgObs, src=Src, dst=Dst, RegridMethod='BILINEAR', UseFiles=False )
    taux_v_yx=RgF.Horz(xfld_Src=taux_v_yx , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
    tauy_v_yx=RgF.Horz(xfld_Src=tauy_v_yx , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
    curltau_v_yx=RgF.Horz(xfld_Src=curltau_v_yx , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
    lat_v,lon_v = GrU.latlon( grid=Dst)


In [ ]:
DateRange = f"{(A.X.time[0].values.item() ).strftime('%Y-%m-%d')}_{(A.X.time[-1].values.item() ).strftime('%Y-%m-%d')}"

In [ ]:
dolev = 21 #np.linspace(-1,1,num=21)
olev = np.linspace(-.5,.5,num=21) * 1.e-6
# Create a figure with a 2x2 grid of subplots
fig, ax = plt.subplots(1, 3, figsize=(20, 5))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()

#GGG
co=ax[0].contourf( Al[0].curltau_s_yx , levels = olev, cmap='bwr' )
cb=plt.colorbar( co )

co=ax[1].contourf( Al[1].curltau_s_yx , levels = olev, cmap='bwr' )
cb=plt.colorbar( co )


co=ax[2].contourf( -1*curltau_v_yx , levels = olev , cmap='bwr' )
cb=plt.colorbar( co )


In [ ]:
#from myPythonTools.Plotting import LatLonMaps as LL 
# Cartopy for pretty maps
#clev=0.5*np.linspace( -0.1,0.1,num=51) 
clev = np.linspace(-.5,.5,num=21) * 1.e-6
cmaps = ['bwr',]  #'gist_ncar','gist_ncar', ] #
fldoo="Wind Stress Curl"


if (cfg['plot']['ForceRegrid']==True):
    fields=[    Al[0].curltau_s_yx , Al[1].curltau_s_yx , -1*curltau_v_yx ,  Al[0].curltau_s_yx+curltau_v_yx , Al[1].curltau_s_yx+curltau_v_yx ,  ]    #,scale*aa_s[:,:]  ]
    lons = [ Al[0].lon_yx , Al[1].lon_yx, lon_v , lon_v , lon_v ]
    lats = [ Al[0].lat_yx , Al[1].lat_yx, lat_v, lat_v, lat_v]
    clevs=[clev,clev,clev , clev, clev,  ] # ,clev,clev,clev/2]
    titles= [ f"{fldoo} {season.upper()} {Al[0].exp} " , f"{fldoo} {season.upper()} {Al[1].exp} " , f"{fldoo} {season.upper()} Large-Yeager" , "diff" , "diff" ]
else:
    fields=[    Al[0].curltau_s_yx , Al[1].curltau_s_yx , -1*curltau_v_yx  ,  ]    #,scale*aa_s[:,:]  ]
    lons = [ Al[0].lon_yx , Al[1].lon_yx, lon_v ,]
    lats = [ Al[0].lat_yx , Al[1].lat_yx, lat_v ,]
    clevs=[clev,clev,clev ,  ] # ,clev,clev,clev/2]
    titles= [ f"{fldoo} {season.upper()} {Al[0].exp}" , f"{fldoo} {season.upper()} {Al[1].exp} " , f"{fldoo} {season.upper()} Large-Yeager" ]



MapProj = ccrs.PlateCarree(central_longitude=180.)

LL.Maps( fields=fields ,lons=lons ,lats=lats ,clevs=clevs,cmaps=cmaps , titles=titles,  Projection=MapProj, Landmask=True )

In [ ]:
###
print( Al[0].exp )
print( Al[1].exp )
